In [1]:
# 已有label，和pred,计算iou阈值下的pred和label
import os

In [2]:
label_mapping = {
            0: 26,
            1: 26,
            2: 26,
            3: 26,
            4: 26,
            5: 26,
            6: 26,
            7: 26,
            8: 25,
            9: 25,
            10: 25,
            11: 25,
            12: 25,
            13: 25,
            14: 25,
            15: 25,
            16: 27,
            17: 27,
            18: 27,
            19: 27,
            20: 27,
            21: 27,
            22: 27,
            23: 27,
            24: 24,
            25: 25,
            26: 26,
            27: 27,
            28: 28,
            29: 29, # Rpedestrian
            30: 30, # Gpedestrian
            31: 31, # Ypedestrian
            32: 32,
            33: 33,
            34: 34,
            35: 35,
            36: 36,
            37: 37,
            38: 38,
            39: 26,
            40: 25,
            41: 27,
            42: 25,
            43: 26,
            44: 25,
            45: 26,
            46: 26,
            47: 25,
            48: 26,
            49: 25
            }

In [3]:
def calculate_iou(box1, box2):
    """
    计算两个矩形框的 Intersection over Union（IoU）
    
    参数：
    box1：(x, y, w, h) 形式的矩形框1坐标，左上角坐标、宽度和高度
    box2：(x, y, w, h) 形式的矩形框2坐标，左上角坐标、宽度和高度
    
    返回：
    IoU：Intersection over Union 值                                               
    """
    # 提取框的坐标和尺寸
    x1_box1, y1_box1, w_box1, h_box1 = box1[1], box1[2],box1[3],box1[4]
    x1_box2, y1_box2, w_box2, h_box2 = box2[1], box2[2],box2[3],box2[4]
    
    # 计算两个框的交集区域的坐标
    x1_intersection = max(x1_box1, x1_box2)
    y1_intersection = max(y1_box1, y1_box2)
    x2_intersection = min(x1_box1 + w_box1, x1_box2 + w_box2)
    y2_intersection = min(y1_box1 + h_box1, y1_box2 + h_box2)
    
    # 计算交集区域的面积
    intersection_area = max(0, x2_intersection - x1_intersection) * max(0, y2_intersection - y1_intersection)
    
    # 计算两个框的面积
    box1_area = w_box1 * h_box1
    box2_area = w_box2 * h_box2
    
    # 计算 IoU
    iou = intersection_area / float(box1_area + box2_area - intersection_area)
    # print("iou没得问题")
    return iou

if __name__ == "__main__":
    # 示例矩形框坐标：(x, y, w, h)
    box1 = [1,50, 50, 100, 100]
    box2 = [1,100, 100, 100, 100]

    iou = calculate_iou(box1, box2)
    print("IoU 值:", iou)
    # print(type(iou))


IoU 值: 0.14285714285714285


In [4]:
#box格式：
# -class
# -x
# -y
# -w
# -h

In [5]:
# 传入：
# gt:list[list]
# pred:list[list]
# iou_threshold:0.45
# label_mapping = dict()
def calcul_tp(gt_boxes, pred_boxes, iou_threshold=0.45, label_mapping = None):
    tp = 0

    for gt in gt_boxes:
        # gt_find = False
        for pred in pred_boxes:

            if label_mapping:
                gt[0] = label_mapping[int(gt[0])]
                pred[0] = label_mapping[int(pred[0])]
            if gt[0] == pred[0]:
                # print("??")
                if calculate_iou(gt,pred) > iou_threshold:
                    tp += 1
                    break
        
    return tp


In [6]:
label_mapping = {
        0: 26,
        1: 26,
        2: 26,
        3: 26,
        4: 26,
        5: 26,
        6: 26,
        7: 26,
        8: 25,
        9: 25,
        10: 25,
        11: 25,
        12: 25,
        13: 25,
        14: 25,
        15: 25,
        16: 27,
        17: 27,
        18: 27,
        19: 27,
        20: 27,
        21: 27,
        22: 27,
        23: 27,
        24: 24,
        25: 25,
        26: 26,
        27: 27,
        28: 28,
        29: 29, # Rpedestrian
        30: 30, # Gpedestrian
        31: 31, # Ypedestrian
        32: 32,
        33: 33,
        34: 34,
        35: 35,
        36: 36,
        37: 37,
        38: 38,
        39: 26,
        40: 25,
        41: 27,
        42: 25,
        43: 26,
        44: 25,
        45: 26,
        46: 26,
        47: 25,
        48: 26,
        49: 25
        }

In [7]:
a = "52 2523.352 24235.423"
if int(a.split(" ")[0]) in {25,26,27}:
    print("gg")

In [9]:
import os
# 计算PR值

def calcul_PR(label_path, pred_path, iou_threshold, conf_threshold, label_mapping):
    label_list = os.listdir(label_path)
    label_cnt = 0
    pred_cnt = 0
    tp = 0
    for file in label_list:
        if not os.path.exists(os.path.join(pred_path, file)):
            print("file:{} not exists!".format(os.path.join(pred_path, file)))
            continue
        
        
        tmp_predbox = []
        tmp_gtbox = []
        with open(os.path.join(label_path, file), "r") as f1:
            for line in f1.readlines():
                """只计算红绿黄"""
                if label_mapping[int(line.split(" ")[0])] not in {25,26,27}:
                    continue


                label_cnt += 1
                tmp_gtbox.append([float(item) for item in line.split(" ")])

        # 预测这里的结构是 class conf x y w h
        print(pred_path,file)
        with open(os.path.join(pred_path, file), "r") as f2:
            for line in f2.readlines():

                """只计算红绿黄"""
                if label_mapping[int(line.split(", ")[0])] not in {25,26,27}:
                    continue
                
                # print(line)
                pred = [float(item) for item in line.split(", ")]
                if pred[1] >= conf_threshold:
                    pred_cnt += 1
                    pred.pop(1)
                    tmp_predbox.append(pred)
        tp += calcul_tp(tmp_gtbox, tmp_predbox, iou_threshold, label_mapping)        
    
    print("算完了，tp有{},gt有{},预测label有{}".format(tp, label_cnt, pred_cnt))
    P = tp/pred_cnt
    R = tp/label_cnt
    return P,R

if __name__ == "__main__":
    label_path = r"Z:\datasets\testlabel\od_labels_tld_0911"
    pred_path = r"Z:\datasets\testlabel\output20230925120329"
    iou_threshold = 0.45 # 这里的iou是判断框对不对
    conf_threshold = 0.6

    
    P, R = calcul_PR(label_path, pred_path, iou_threshold, conf_threshold, label_mapping)
    print("最后的结果P：{:.4f}, R：{:.4f}".format(P,R))

Z:\datasets\testlabel\output20230925120329 1.txt
Z:\datasets\testlabel\output20230925120329 10.txt
Z:\datasets\testlabel\output20230925120329 100.txt
Z:\datasets\testlabel\output20230925120329 1000.txt
Z:\datasets\testlabel\output20230925120329 1001.txt
Z:\datasets\testlabel\output20230925120329 1002.txt
Z:\datasets\testlabel\output20230925120329 1003.txt
Z:\datasets\testlabel\output20230925120329 1004.txt
Z:\datasets\testlabel\output20230925120329 1005.txt
Z:\datasets\testlabel\output20230925120329 1006.txt
Z:\datasets\testlabel\output20230925120329 1007.txt
Z:\datasets\testlabel\output20230925120329 1008.txt
Z:\datasets\testlabel\output20230925120329 1009.txt
Z:\datasets\testlabel\output20230925120329 101.txt
Z:\datasets\testlabel\output20230925120329 1010.txt
Z:\datasets\testlabel\output20230925120329 1011.txt


KeyError: 54

In [24]:
a = 42.423
print("fsd{:.4f}".format(a))

fsd42.4230


In [27]:
a = [*(5)]
a

TypeError: Value after * must be an iterable, not int

In [52]:
with open(os.path.join(r"Z:\datasets\lbmap\tld_label", "16.txt"), "r") as f1:
    for line in f1.readlines():
        print([float(item) for item in line.split(" ")])

[35.0, 0.1558, 0.385914, 0.004979, 0.01112]
[35.0, 0.192267, 0.381028, 0.005744, 0.012482]
[35.0, 0.232344, 0.377977, 0.006123, 0.01443]
[36.0, 0.096127, 0.39514, 0.006074, 0.009988]
[36.0, 0.126748, 0.391175, 0.005389, 0.010814]
[36.0, 0.020236, 0.408891, 0.005618, 0.010798]
[36.0, 0.068306, 0.400475, 0.005315, 0.010258]
[36.0, 0.042326, 0.4049, 0.005163, 0.009448]
[36.0, 0.330926, 0.367371, 0.007551, 0.013326]
[36.0, 0.378944, 0.363977, 0.007811, 0.013785]
[35.0, 0.427455, 0.361791, 0.009711, 0.01616]


In [28]:
a = [1,2,3,4,5]
a.pop(1)
a

[1, 3, 4, 5]

In [12]:
35.0 == int(35)

True